##1.1

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('TSLA.csv')
print(df.head())
print(df.tail())
print(df.describe())

           Date  Close Price  ...  Low Price  Total Traded Quantity
0  Oct 24, 2019       299.68  ...     290.05                 29.82M
1  Oct 23, 2019       254.68  ...     251.35                 11.22M
2  Oct 22, 2019       255.58  ...     250.85                  4.63M
3  Oct 21, 2019       253.50  ...     250.18                  5.11M
4  Oct 18, 2019       256.95  ...     255.10                  5.75M

[5 rows x 6 columns]
             Date  Close Price  ...  Low Price  Total Traded Quantity
498  Oct 31, 2017       331.53  ...     320.18                  5.67M
499  Oct 30, 2017       320.08  ...     317.25                  4.25M
500  Oct 27, 2017       320.87  ...     316.66                  6.98M
501  Oct 26, 2017       326.17  ...     323.20                  5.02M
502  Oct 25, 2017       325.84  ...     323.56                  8.59M

[5 rows x 6 columns]
       Close Price  Open Price  High Price   Low Price
count   503.000000  503.000000  503.000000  503.000000
mean    291.988330

##1.2

In [ ]:
df_90=df[0:90]
x=df_90.describe()
print("Mean :", x["Close Price"][1])
print("Min :", x["Close Price"][3])
print("Max :", x["Close Price"][7])

Mean : 237.01055555555556
Min : 211.4
Max : 299.68


##1.3

In [ ]:
df['Date']= pd.to_datetime(df['Date'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Date                   503 non-null    datetime64[ns]
 1   Close Price            503 non-null    float64       
 2   Open Price             503 non-null    float64       
 3   High Price             503 non-null    float64       
 4   Low Price              503 non-null    float64       
 5   Total Traded Quantity  503 non-null    object        
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 23.7+ KB


In [ ]:
print(df['Date'][0]-df['Date'][len(df)-1])

729 days 00:00:00


##1.4

In [ ]:
df['month_year'] = pd.to_datetime(df['Date']).dt.to_period('M')
df['Total Traded Quantity'] = df['Total Traded Quantity'].str.rstrip('M')
df['Total Traded Quantity']= pd.to_numeric(df['Total Traded Quantity'])
df['Total Traded Quantity'] = df['Total Traded Quantity']*(1000000)
df["Total Price"] = df["Close Price"]*df["Total Traded Quantity"]

sum_price = df.groupby(df['month_year'])['Total Price'].sum()
sum_volume = df.groupby(df['month_year'])['Total Traded Quantity'].sum()

vwap=sum_price/sum_volume
print("VWAP :",vwap)

VWAP : month_year
2017-10    325.012347
2017-11    309.500577
2017-12    324.671030
2018-01    338.192926
2018-02    333.128621
2018-03    305.362653
2018-04    288.258470
2018-05    289.492634
2018-06    337.848719
2018-07    313.085949
2018-08    338.429717
2018-09    283.920060
2018-10    290.774345
2018-11    345.132018
2018-12    343.673861
2019-01    312.929452
2019-02    307.989824
2019-03    278.525216
2019-04    262.122016
2019-05    217.257468
2019-06    211.260398
2019-07    241.165854
2019-08    224.566217
2019-09    237.005077
2019-10    256.960142
Freq: M, dtype: float64


##1.5

In [ ]:
def avg_N(N):
  df_N=df[0:N]
  t=df_90.describe()
  print("Avg :", t["Close Price"][1])

avg_N(90)

Avg : 237.01055555555556


In [ ]:
def profit_loss(N):
  pl=list(df['Close Price'])
  return ((pl[0]-pl[N-1]) / pl[N-1])*100

profit_loss(90)

32.349953628052816

##1.6


In [ ]:
df_temp = df["Close Price"]
df_temp = df_temp[::-1]
df_temp = df_temp.pct_change().fillna(0)
df_temp = (df_temp[::-1])*(100)
print(df_temp)

0      17.669232
1      -0.352140
2       0.820513
3      -1.342674
4      -1.916250
         ...    
498     3.577231
499    -0.246206
500    -1.624920
501     0.101277
502     0.000000
Name: Close Price, Length: 503, dtype: float64


In [ ]:
df['day_perc_change']=df_temp
print(df.head())


        Date  Close Price  Open Price  ...  month_year   Total Price  day_perc_change
0 2019-10-24       299.68      298.97  ...     2019-10  8.936458e+09        17.669232
1 2019-10-23       254.68      254.50  ...     2019-10  2.857510e+09        -0.352140
2 2019-10-22       255.58      254.32  ...     2019-10  1.183335e+09         0.820513
3 2019-10-21       253.50      258.33  ...     2019-10  1.295385e+09        -1.342674
4 2019-10-18       256.95      260.70  ...     2019-10  1.477462e+09        -1.916250

[5 rows x 9 columns]


##1.7

In [ ]:
def l_df(x):
  res=''
  if(x < 0.5 and x>=-0.5):
    res='Slight or no change'
  elif(x < 1 and x>=0.5):
    res='Slight positive'
  elif(x < -0.5 and x>=-1):
    res='Slight negative'
  elif(x < 3 and x>=1):
    res='Positive'
  elif(x < -1 and x>=-3):
    res='Nositive'
  elif(x < 7 and x>=3):
    res='Among top gainers'
  elif(x < -3 and x>=-7):
    res='Among top losers'
  elif(x > 7):
    res='Bull run'
  else:
    res='Bear drop'
  return res


df['Trend'] = df['day_perc_change'].apply(lambda x: l_df(x))
print(df['Trend'][79])

Among top gainers


##1.8

In [ ]:
mean = df.groupby(df['Trend'])['Total Traded Quantity'].mean()
median = df.groupby(df['Trend'])['Total Traded Quantity'].median()
print(mean,median)

Trend
Among top gainers      1.100100e+07
Among top losers       1.125907e+07
Bear drop              1.839286e+07
Bull run               2.004364e+07
Nositive               7.830693e+06
Positive               7.134673e+06
Slight negative        6.668293e+06
Slight or no change    6.793011e+06
Slight positive        7.209688e+06
Name: Total Traded Quantity, dtype: float64 Trend
Among top gainers       9900000.0
Among top losers        9970000.0
Bear drop              18450000.0
Bull run               19900000.0
Nositive                7000000.0
Positive                6840000.0
Slight negative         5840000.0
Slight or no change     6190000.0
Slight positive         6570000.0
Name: Total Traded Quantity, dtype: float64


##1.9

In [ ]:
df.to_csv('week2.csv')